In [26]:
from sklearn.linear_model            import LogisticRegression
from sklearn.model_selection         import GridSearchCV, train_test_split, cross_val_score
from sklearn.metrics                 import accuracy_score, recall_score, roc_auc_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag                      import Okt; t=Okt()
import warnings
warnings.filterwarnings(action='ignore')

In [27]:
from jamo import h2j, j2hcj

def jamo_tokenizer(text):
    return j2hcj(h2j(text))

In [45]:
train = pd.read_csv('./new_train.csv')
dev = pd.read_csv('../../data/dev.hate.csv')
test = pd.read_csv('../../data/test.hate.no_label.csv')
train.head()

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate


In [46]:
test.head()

,comments
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ
1,둘다 넘 좋다~행복하세요
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요
4,장현승 얘도 참 이젠 짠하다...


In [47]:
train = pd.concat([train, dev], axis=0)
train.head()

,comments,label
0,(현재 호텔주인 심정) 아18 난 마른하늘에 날벼락맞고 호텔망하게생겼는데 누군 계속...,hate
1,....한국적인 미인의 대표적인 분...너무나 곱고아름다운모습...그모습뒤의 슬픔을...,none
2,"...못된 넘들...남의 고통을 즐겼던 넘들..이젠 마땅한 처벌을 받아야지..,그래...",hate
3,"1,2화 어설펐는데 3,4화 지나서부터는 갈수록 너무 재밌던데",none
4,1. 사람 얼굴 손톱으로 긁은것은 인격살해이고2. 동영상이 몰카냐? 메걸리안들 생각...,hate


In [48]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8367 entries, 0 to 470
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comments  8367 non-null   object
 1   label     8367 non-null   object
dtypes: object(2)
memory usage: 196.1+ KB


In [49]:
# 특수문자 제거
import re

def cleanse(text):
    pattern = re.compile(r'\s+')
    text = re.sub(pattern, ' ', text)
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9]', ' ', text)
    return text

train['comments'] = train['comments'].apply(cleanse)

In [50]:
# 띄어쓰기
from pykospacing import spacing
train['comments'] = train['comments'].apply(spacing)

In [51]:
# 문장 분리
import kss
train['comments'] = train['comments'].apply(kss.split_sentences)
train['comments'] = [','.join(map(str, ls)) for ls in train['comments']]

In [52]:
# 중복 제거
from soynlp.normalizer import *
train['comments'] = [repeat_normalize(comment, num_repeats=2) for comment in train['comments']]

In [53]:
X_train = train['comments']
X_test = test['comments']
y_train = train['label']
# y_test = test['label']

# 벡터라이저 생성
tfidf_vect = TfidfVectorizer(tokenizer=jamo_tokenizer, ngram_range=(1,6), min_df=3, max_df=0.99)

# 벡터라이저 학습
tfidf_vect.fit(X_train)

# 벡터라이저 리턴
tfidf_matrix_train = tfidf_vect.transform(X_train)

# 로지스틱 리그레션 학습
lr = LogisticRegression(multi_class='multinomial', class_weight='balanced')
lr.fit(tfidf_matrix_train, y_train)

# 로지스틱 리그레션 예측
tfidf_matrix_test = tfidf_vect.transform(X_test)
preds = lr.predict(tfidf_matrix_test)
len(preds)
# accuracy_score(y_test, preds), f1_score(y_test, preds, average='macro')

974

In [54]:
test['label'] = preds
test.head()

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,none
1,둘다 넘 좋다~행복하세요,none
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,offensive
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,none
4,장현승 얘도 참 이젠 짠하다...,none


In [55]:
test['label'][test['label'] == 'none'] = 0

In [56]:
def kaggle_format(df):
    df['label'][df['label'] == 'none'] = 0
    df['label'][df['label'] == 'offensive'] = 1
    df['label'][df['label'] == 'hate'] = 2
    return df

In [57]:
kaggle_format(test)

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,0
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,1
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,0
...,...,...
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,0
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,2
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,0
972,입에 손가릭이 10개 있으니 징그럽다,2


In [58]:
test.head()

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,0
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,1
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,0


In [59]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 974 entries, 0 to 973
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   comments  974 non-null    object
 1   label     974 non-null    object
dtypes: object(2)
memory usage: 15.3+ KB


In [60]:
test.to_csv('./0110_jc.csv', index=False)

In [61]:
pd.read_csv('./0110_jc.csv')

,comments,label
0,ㅋㅋㅋㅋ 그래도 조아해주는 팬들 많아서 좋겠다 ㅠㅠ 니들은 온유가 안만져줌 ㅠㅠ,0
1,둘다 넘 좋다~행복하세요,0
2,근데 만원이하는 현금결제만 하라고 써놓은집 우리나라에 엄청 많은데,1
3,원곡생각하나도 안나고 러블리즈 신곡나온줄!!! 너무 예쁘게 잘봤어요,0
4,장현승 얘도 참 이젠 짠하다...,0
...,...,...
969,대박 게스트... 꼭 봐야징~ 컨셉이 바뀌니깐 재미지넹,0
970,성형으로 다 뜯어고쳐놓고 예쁜척. 성형 전 니 얼굴 다 알고있다. 순자처럼 된장냄새...,2
971,분위기는 비슷하다만 전혀다른 전개던데 무슨ㅋㅋㄱ 우리나라사람들은 분위기만 비슷하면 ...,0
972,입에 손가릭이 10개 있으니 징그럽다,2
